In [1]:
import numpy as np
from collections import namedtuple
import codecs
from hmm_tools import *

# ПММ. Частина 3. Декодування.

У цій частині практичного завдання я хочу змоделювати процес перехоплення та розшифровки ворожих радіопередач. 
Для цього би ідеально підійшов текст твору "Криптономікон" Ніла Стівенсона, але його ще не переклали українською. 
Тому продовжую використовувати текст з минулих частин роботи.

### Передумови:  
- Відомо, що шифр є простим шифром підстановки, який ставить у відповідність кожному символу алфавіту відкритого тексту символ алфавіту зашифрованого тексту
- Алфавіти відкритого тексту та шифровки збігаються (з точки зору математики важливо лише, що вони одного розміру)
- Є деякий об'єм попередньо перехоплених повідомлень і наступні повідомлення

### Математична модель:  
Розшифровувати будемо за допомогою ПММ із 33-ма прихованими станами та 33-ма можливими спостереженнями. 
Приховані стани відповідатимуть літерам відкритого тексту, спостереження - літерам шифротексту.
- Матриця $A$ буде сталою та показуватиме частоти біграмів (ймовірності переходу від однієї літери до іншої).
- Матриця $B$ буде відповідати власне за розшифровку і саме її ми будемо оптимізовувати.
- Після оптимізації необхідно буде знайти оптимальну послідовність прихованих станів

### План роботи:  

1. Алгоритм Баума-Велша для навчання ПММ на шифрованих даних (власне, пошук оптимального $B$)
2. Алгоритм Вітербі для пошуку відкритого тексту, що максимізує ймовірність отримання даного шифротексту за даними $\mu, A, B$

## 3.1. Підготовка тренувальних та тестових тексту та шифротексту

In [2]:
N_TRAIN = 100000
N_TEST = 50000

FILE = "1251.full.peter-watts-starfish.txt"

raw_train_data = read_textfile(FILE, N_TRAIN)
raw_test_data = read_textfile(FILE, N_TEST, char_offset=N_TRAIN)

Offset done. Starting reading actual data.
read checkpoint 10000 (13488)
read checkpoint 20000 (26687)
read checkpoint 30000 (40061)
read checkpoint 40000 (53637)
read checkpoint 50000 (67100)
read checkpoint 60000 (80630)
read checkpoint 70000 (94363)
read checkpoint 80000 (107347)
read checkpoint 90000 (121016)
read checkpoint 100000 (134458)
read - end
offset checkpoint 90000 (13488)
offset checkpoint 80000 (26687)
offset checkpoint 70000 (40061)
offset checkpoint 60000 (53637)
offset checkpoint 50000 (67100)
offset checkpoint 40000 (80630)
offset checkpoint 30000 (94363)
offset checkpoint 20000 (107347)
offset checkpoint 10000 (121016)
offset checkpoint 0 (134458)
Offset done. Starting reading actual data.
read checkpoint 10000 (147577)
read checkpoint 20000 (161070)
read checkpoint 30000 (174559)
read checkpoint 40000 (188116)
read checkpoint 50000 (201811)
read - end


In [3]:
train_data = text2states(raw_train_data, len(raw_train_data))
test_data = text2states(raw_test_data, len(raw_test_data))
print(train_data[:30], len(train_data))
print(test_data[:30], len(test_data))

[19  0 22  9 20  6 18 22 22 21 16 18 20 21 30 14  4 11  0 20 14  4 19 20
  9 15 31  8  0 32] 100001
[ 6 17 12 26 22  6  4 18  8 17  4 11 17 12 25 11  8  4  6  4 15  4 21 32
 16  4 13 10  9 17] 50001


In [4]:
def freqs(data):
    mu = np.zeros(33, np.uint32)
    A = np.zeros((33,33), np.uint32)
    prev_c = None
    for c in data:
        mu[c] = mu[c] + 1
        if prev_c is not None:
            A[prev_c, c] = A[prev_c, c] + 1
        prev_c = c
    return mu, A

train_mu, train_A = freqs(train_data)
train_mu += 5
train_A += 5
print("Freqs:", train_mu)
print("Bigrams:", train_A)

Freqs: [5059   25 1219  760 8697 1814 4927 1572 3377 5116 1009 2470 6095 1132
 3903 4066 2989 6503 9035 2713 4473 4073 5459 3382  194  943  768 1390
  869  617 1937 1099 2481]
Bigrams: [[ 22   5  98 ...   5  37  90]
 [  6   5   5 ...   5   5   5]
 [ 18   5   5 ...   5  29  38]
 ...
 [ 47   6   7 ...   5   5  38]
 [ 12   5  69 ...   5  38  27]
 [ 26   5 101 ...   5  28  33]]


In [5]:
train_mu = train_mu.astype(np.double) / train_mu.sum()
train_A = train_A.astype(np.double) / train_A.sum(axis=1, keepdims=True)

### Шифрування

In [6]:
def encode(plaintext, rot):
    return np.mod(plaintext + rot, 33)

def decode(cyphertext, rot):
    return encode(cyphertext, -rot)

In [7]:
train_encoded = encode(train_data, 13)
test_encoded = encode(test_data, 13)
print(train_encoded[:30])
print(test_encoded[:30])

[32 13  2 22  0 19 31  2  2  1 29 31  0  1 10 27 17 24 13  0 27 17 32  0
 22 28 11 21 13 12]
[19 30 25  6  2 19 17 31 21 30 17 24 30 25  5 24 21 17 19 17 28 17  1 12
 29 17 26 23 22 30]


### Алгоритм Вітербі

In [8]:
Viterbi_res = namedtuple('Viterbi_res', 'X, prob')

def Viterbi(Mu, A, B, Y, eps=1e-10):
    N = Mu.shape[0]
    M = B.shape[1]
    T = Y.shape[0]

    # Preprocessing
    # logMu = np.empty_like(Mu)
    # logMu[Mu <= eps] = -np.inf 
    # logMu[Mu > eps] = np.log(Mu[Mu>eps])
    # logA = np.empty_like(A)
    # logA[A <= eps] = -np.inf
    # logA[A > eps] = np.log(A[A>eps])
    # logB = np.empty_like(B)
    # logB[B <= eps] = -np.inf
    # logB[B > eps] = np.log(B[B>eps])

    logMu = np.log(Mu)
    logA = np.log(A)
    logB = np.log(B)

    print("Mu", logMu)
    print("A", logA)
    print("B", logB)

    # q = best states
    q = np.full_like(Y, -1, np.int8)
    # psi - for backtracking
    psi = np.zeros((T, N), np.int8)

    # d : 1xN
    d = logMu + logB[:,Y[0]]
    if d.shape != (N,): raise ValueError(f"d shape is {d.shape}")
    psi[0] = -1

    for t in range(1,T):
        U = d.reshape(-1,1) + logA
        psi[t] = np.argmax(U, axis=0)
        d = logB[:, Y[t]] + np.max(U, axis=0)
        if d.shape != (N,): raise ValueError(f"d shape is {d.shape}")

    print("d:", d)

    q[T-1] = np.argmax(d)
    res_p = d[q[T-1]]

    for t in range(T-2, -1, -1):
        q[t] = psi[t+1, q[t+1]]
    
    return Viterbi_res(q, res_p)


## 3.2. 
Беремо A з тренувального тексту. Запускаємо алгоритм Баума-Велша на перевірочному тексті.

In [9]:
params = BaumWelch_params(
    max_iter=200, eps=1e-7, 
    n_hidden_states=33, n_observation_states=33, n_observations=50000,
    const_Mu=False, const_A=True, const_B=False
)
init_vals = BaumWelch_init(
    Mu=None, 
    A=train_A, 
    B=None
)
bw_res = learn(test_encoded, params, init_vals)
print(bw_res)


init c: [32.99904248 32.99980538 33.00099669 ... 33.0038407  33.00042341
 32.99838617]
Starting Baum-Welch process. Initial probability = -174825.11738376034
Initial Mu: [0.03030152 0.03031898 0.03031236 0.03030832 0.0302949  0.0302949
 0.03029193 0.03031642 0.03030839 0.03031163 0.0302908  0.03031957
 0.0303154  0.03029661 0.03029568 0.03029573 0.03029939 0.03030608
 0.03030326 0.030299   0.03030871 0.0302944  0.03029903 0.03030128
 0.03030399 0.03031397 0.03029622 0.03030576 0.03030813 0.03029158
 0.03030858 0.03029534 0.03029214]
Initial A: [[0.00421537 0.00095804 0.01877754 ... 0.00095804 0.00708948 0.01724468]
 [0.03243243 0.02702703 0.02702703 ... 0.02702703 0.02702703 0.02702703]
 [0.01305294 0.00362582 0.00362582 ... 0.00362582 0.02102973 0.0275562 ]
 ...
 [0.02241297 0.00286123 0.0033381  ... 0.00238436 0.00238436 0.01812113]
 [0.00953137 0.00397141 0.0548054  ... 0.00397141 0.03018268 0.02144559]
 [0.00984476 0.00189322 0.03824309 ... 0.00189322 0.01060204 0.01249527]]
Initia

In [10]:
print(bw_res.Mu)
print(bw_res.A)
print(bw_res.B)

[0.00000000e+000 0.00000000e+000 7.87238151e-287 6.97018559e-284
 0.00000000e+000 0.00000000e+000 1.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 2.08233978e-276 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000 1.02923273e-199 0.00000000e+000 0.00000000e+000
 0.00000000e+000 0.00000000e+000 0.00000000e+000 0.00000000e+000
 0.00000000e+000]
[[0.00421537 0.00095804 0.01877754 ... 0.00095804 0.00708948 0.01724468]
 [0.03243243 0.02702703 0.02702703 ... 0.02702703 0.02702703 0.02702703]
 [0.01305294 0.00362582 0.00362582 ... 0.00362582 0.02102973 0.0275562 ]
 ...
 [0.02241297 0.00286123 0.0033381  ... 0.00238436 0.00238436 0.01812113]
 [0.00953137 0.00397141 0.0548054  ... 0.00397141 0.03018268 0.02144559]
 [0.00984476 0.00189322 0.03824309 ... 0.00189322 0.01060204 0.01249527]]
[[1.21967912e-159 

## 3.3
Тепер запускаємо алгоритм Вітербі щоб знайти оптимальний відкритий текст для даного закритого тексту та за знайденою раніше матрицею B

In [11]:
vb_res = Viterbi(bw_res.Mu, bw_res.A, bw_res.B, test_encoded, 1e-40)
s = score(test_data, vb_res.X)
print("best p = ", vb_res.prob)
print("="*40)
print("=== score is ", s, "===")
print("="*40)
print(vb_res.X.shape)

Mu [         -inf          -inf -658.77856107 -651.99252456          -inf
          -inf    0.                  -inf          -inf          -inf
          -inf          -inf          -inf          -inf          -inf
          -inf -634.77999351          -inf          -inf          -inf
          -inf          -inf          -inf          -inf          -inf
 -458.1856199           -inf          -inf          -inf          -inf
          -inf          -inf          -inf]
A [[-5.46901864 -6.95062318 -3.97509361 ... -6.95062318 -4.94914318
  -4.06025142]
 [-3.42859636 -3.61091791 -3.61091791 ... -3.61091791 -3.61091791
  -3.61091791]
 [-4.33874212 -5.61967597 -5.61967597 ... -5.61967597 -3.86181805
  -3.59152772]
 ...
 [-3.79811543 -5.85650356 -5.70235288 ... -6.03882512 -6.03882512
  -4.01067687]
 [-4.65316638 -5.52863512 -2.90396653 ... -5.52863512 -3.50048687
  -3.84223617]
 [-4.62081637 -6.269475   -3.2637924  ... -6.269475   -4.5467084
  -4.38240535]]
B [[-365.91244198 -227.91118188 -1

C:\Users\mkrooted\AppData\Local\Temp\ipykernel_19472\4103319145.py:19: RuntimeWarning: divide by zero encountered in log
  logMu = np.log(Mu)
C:\Users\mkrooted\AppData\Local\Temp\ipykernel_19472\4103319145.py:21: RuntimeWarning: divide by zero encountered in log
  logB = np.log(B)


d: [-134978.95961646 -134889.86267786 -134918.08510254 -134867.55907175
 -134974.33032664 -134869.94758331 -134863.45075362 -134872.03448437
 -134860.30874125 -134973.4007275  -134853.59122623 -134847.86861639
 -135002.95521042 -134886.80694202 -134855.87038886 -134873.75218366
 -134860.03474371 -134864.21564488 -134957.31082478 -134882.24387625
 -134924.60466877 -134881.09096328 -134872.41714848 -134976.87985125
 -134883.13028589 -134856.6718376  -134906.74188312 -134879.30531866
 -134908.86739434 -134914.92695686 -135086.5006229  -134905.53388638
 -134946.52859532]
best p =  -134847.8686163909
=== score is  0.996580068398632 ===
(50001,)


**Кількість розшифрованих символів - 99%!**

In [12]:
text_out = "".join(list(np.array(list("іґєїабвгдежзийклмнопрстуфхцчшщьюя"))[test_data[:200]]))
print(text_out)

decoded_out = "".join(list(np.array(list("іґєїабвгдежзийклмнопрстуфхцчшщьюя"))[vb_res.X[:200]]))
print(decoded_out)

вництваодназнихздаваласямайженормальноютеревенячитажартуючивонаажзішкірипнуласянібинамагаючисьякоськомпенсуватитойфактщозовнівонаскидаласяназомбіджоелзабувїїімядругазавсюдорогуназронилаанісловаодинізт
вництваодназнихздаваласямайженормальноютеревенячитажартуючивонаажзішкірипнуласянібинамагаючисьякоськомпенсуватитойфактщозовнівонаскидаласяназомбіджоелзабувїїімядругазавсюдорогуназронилаанісловаодинізт


### 3.4 На всякий випадок перевірю ці ж матриці на зовсім іншій частині книги

In [30]:
raw_test_data2 = read_textfile(FILE, 100000, char_offset=500000)
test_data2 = text2states(raw_test_data2, len(raw_test_data2))

offset checkpoint 490000 (13488)
offset checkpoint 480000 (26687)
offset checkpoint 470000 (40061)
offset checkpoint 460000 (53637)
offset checkpoint 450000 (67100)
offset checkpoint 440000 (80630)
offset checkpoint 430000 (94363)
offset checkpoint 420000 (107347)
offset checkpoint 410000 (121016)
offset checkpoint 400000 (134458)
offset checkpoint 390000 (147577)
offset checkpoint 380000 (161070)
offset checkpoint 370000 (174559)
offset checkpoint 360000 (188116)
offset checkpoint 350000 (201811)
offset checkpoint 340000 (215730)
offset checkpoint 330000 (229493)
offset checkpoint 320000 (243303)
offset checkpoint 310000 (256893)
offset checkpoint 300000 (270718)
offset checkpoint 290000 (284592)
offset checkpoint 280000 (298127)
offset checkpoint 270000 (311809)
offset checkpoint 260000 (325698)
offset checkpoint 250000 (338821)
offset checkpoint 240000 (351926)
offset checkpoint 230000 (365414)
offset checkpoint 220000 (378516)
offset checkpoint 210000 (391954)
offset checkpoint 200

In [31]:
test_data2_encoded = encode(test_data2, 13)
test_data2_encoded.shape

(100001,)

In [32]:
vb_res = Viterbi(bw_res.Mu, bw_res.A, bw_res.B, test_data2_encoded, 1e-40)
s = score(test_data2, vb_res.X)
print("best p = ", vb_res.prob)
print("="*40)
print("=== score is ", s, "===")
print("="*40)
print(vb_res.X.shape)

C:\Users\mkrooted\AppData\Local\Temp\ipykernel_19472\4103319145.py:19: RuntimeWarning: divide by zero encountered in log
  logMu = np.log(Mu)
C:\Users\mkrooted\AppData\Local\Temp\ipykernel_19472\4103319145.py:21: RuntimeWarning: divide by zero encountered in log
  logB = np.log(B)


Mu [         -inf          -inf -658.77856107 -651.99252456          -inf
          -inf    0.                  -inf          -inf          -inf
          -inf          -inf          -inf          -inf          -inf
          -inf -634.77999351          -inf          -inf          -inf
          -inf          -inf          -inf          -inf          -inf
 -458.1856199           -inf          -inf          -inf          -inf
          -inf          -inf          -inf]
A [[-5.46901864 -6.95062318 -3.97509361 ... -6.95062318 -4.94914318
  -4.06025142]
 [-3.42859636 -3.61091791 -3.61091791 ... -3.61091791 -3.61091791
  -3.61091791]
 [-4.33874212 -5.61967597 -5.61967597 ... -5.61967597 -3.86181805
  -3.59152772]
 ...
 [-3.79811543 -5.85650356 -5.70235288 ... -6.03882512 -6.03882512
  -4.01067687]
 [-4.65316638 -5.52863512 -2.90396653 ... -5.52863512 -3.50048687
  -3.84223617]
 [-4.62081637 -6.269475   -3.2637924  ... -6.269475   -4.5467084
  -4.38240535]]
B [[-365.91244198 -227.91118188 -1

In [33]:
text_out = "".join(list(np.array(list("іґєїабвгдежзийклмнопрстуфхцчшщьюя"))[test_data2[:200]]))
print(text_out)

decoded_out = "".join(list(np.array(list("іґєїабвгдежзийклмнопрстуфхцчшщьюя"))[vb_res.X[:200]]))
print(decoded_out)

надимаючисьдочеревногоілюмінатораскафапростопередочимаджоелакаламутнеокеанськесвітлоставалодедаліяскравішимблакитьзміниласязеленнюатодіжовтизноюатодібілиноюутихомуокеанірозверзласядіразїїцентрузійшлос
вадимаючисьдочеревногоілюмінатораскафапростопередочимаджоелакаламутнеокеанськесвітлоставалодедаліяскравішимблакитьзміниласязеленнюатодіжовтизноюатодібілиноюутихомуокеанірозверзласядіразїїцентрузійшлос


**І знову висока точність**

Подивімось на елементи матриці B, більші за $0.001$

In [39]:
with np.printoptions(threshold=np.inf):
    print((bw_res.B > 01e-3).astype(np.uint8))

[[0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 1 0 0 0 0 0 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 

Добре видно діагональ, що дійсно відповідає зсуву на 13 символів, що використовувався при кодуванні 